### Open otu tables and merge them with metadata file and remove 0 read samples, remove some of the Otus which taxonomicaly are not fine.

---

In [2]:

#setworking directory to AbioticAraMicrobe/
setwd('../')

In [3]:
#load packages
library(dplyr)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [12]:
dir.create("data/DataClean")

#### Bacterial table preprocessing


In [17]:
#---------------------------------------------------------------------------------------------
#Bacteria
Otu = read.table("data/MothurOutput/BV5Otu.txt", header = T , check.names = F , stringsAsFactors = F)
taxa = read.table("data/MothurOutput/BV5Taxa.txt", header = T , check.names = F , stringsAsFactors = F)
taxa$Taxonomy = gsub("\\s*\\([^\\)]+\\)","",as.character(taxa$Taxonomy))
otu_taxa <- taxa %>% separate("Taxonomy" , c("Kingdom" , "Phylum" , "Class" , "Order" , "Family" , "Genus" ,"Species" , "Strain" ) , ";" , remove = FALSE)
Unclassifiedotus = subset(otu_taxa,otu_taxa$Kingdom=="Eukaryota" | otu_taxa$Phylum=="Bacteria_unclassified" | otu_taxa$Family=="Mitochondria") #remove all the otus tat are not classified in phylum level from taxonomy table
Otu1 = Otu[,!(colnames(Otu) %in% Unclassifiedotus$OTU)] #remove remove all the otus tat are not classified in phylum level from otu table
sampleinfo <- read.table("data/metadata.txt" , header = T , check.names = F , stringsAsFactors = F) # mapping file containes informatio about data, lab part
#---------------------------------------------------------------------------------------------
Otu2 <- Otu1%>%separate("Group" , c("Loci" , "LibM") , remove = FALSE)
Otu2 = merge(Otu2,sampleinfo[c("Lib","Samplenumber")] , by.x = "LibM" , by.y = "Lib")
sam_otu <- Otu2[,-c(1:5)]
sam_otu1 = aggregate(. ~  Samplenumber, data = sam_otu, sum)  #some samples are repeted in the lab because of low concentration of DNA so we merged them
#sample
sample = sampleinfo[!duplicated(sampleinfo$Samplenumber) , ]  # now remoe duplate sample names
rownames(sample) <- sample$Samplenumber
rownames(sam_otu1) <- sam_otu1$Samplenumber
sam_otu2 = sam_otu1[-1]
#select and save 0 read samples
sam_lessReads = sam_otu2[rowSums(sam_otu2)==0,]
sam_lessReads1 = merge(x =sam_lessReads[,c(1,ncol(sam_lessReads))] , y=sample , by.x="row.names",by.y="row.names")
sam_lessReads1  = sam_lessReads1[,-c(2,2)]
#remove 0 read samples
sam_otu2 = sam_otu2[!rowSums(sam_otu2)==0,] # use this table for Ra log10 and raw reads saving info
#save abundance tabke with samples using sam_otu2
sample1 = sample[(sample$Samplenumber %in% rownames(sam_otu2)),]
rownames(sample1) <- sample1$Samplenumber
####
sam_otu3 = cbind(TotalReads=rowSums(sam_otu2),numOTUs=ncol(sam_otu2),sam_otu2)
cleansam_otu = merge(x = sample1,y = sam_otu3 , by= "row.names")
cleansam_otu = cleansam_otu[-1] ##after removing unclassified otu st phylum level and samples with less than 100 reads
cleansam_otu = cleansam_otu[order(cleansam_otu$TotalReads,decreasing = TRUE),]
taxonomy = otu_taxa[(otu_taxa$OTU %in% colnames(sam_otu3)),]
taxonomy = taxonomy[-ncol(taxonomy)]
taxonomy$Species = "unknown"
write.table(cleansam_otu,file = "Data/DataClean/BV5Otu.txt",quote = FALSE,sep = "\t",row.names = FALSE)
write.table(taxonomy,file = "Data/DataClean/BV5Taxa.txt",quote = FALSE , sep = "\t" , row.names = FALSE)
#write.table(sam_lessReads1,file = "Data/DataClean/BV5readsamples.txt",quote = FALSE , sep = "\t" , row.names = FALSE)
#write.table(Unclassifiedotus,file = "Data/DataClean/BV5Unclaasified.txt",quote = FALSE , sep = "\t" , row.names = FALSE)
print('done!')

Warning message:
“Expected 8 pieces. Missing pieces filled with `NA` in 5621 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


[1] "done!"


#### Eucaryotes table preprocessing

In [16]:
#------------------------------------------------------------------------------------------------
#PrV9
Otu = read.table("data/MothurOutput/PrV9Otu.txt", header = T , check.names = F , stringsAsFactors = F)
taxa = read.table("data/MothurOutput/PrV9Taxa.txt", header = T , check.names = F , stringsAsFactors = F)
taxa$Taxonomy = gsub("\\s*\\([^\\)]+\\)","",as.character(taxa$Taxonomy))
otu_taxa <- taxa %>% separate("Taxonomy" , c("Domain","Kingdom" , "Phylum" , "Class" , "Order" , "Family" , "Genus" ,"Species" , "Strain" ) , ";" , remove = FALSE)
Unclassifiedotus = subset(otu_taxa, Phylum=="Metazoa"  | Phylum=="Fungi" | Kingdom=="Eukaryota_unclassified") #remove all the otus tat are not classified in phylum level from taxonomy table
Otu1 = Otu[,!(colnames(Otu) %in% Unclassifiedotus$OTU)] #remove remove all the otus tat are not classified in phylum level from otu table
sampleinfo <- read.table("data/metadata.txt" , header = T , check.names = F , stringsAsFactors = F) # mapping file containes informatio about data, lab part
#---------------------------------------------------------------------------------------------

Otu2 <- Otu1%>%separate("Group" , c("Loci" , "LibM") , remove = FALSE)
Otu2 = merge(Otu2,sampleinfo[c("Lib","Samplenumber")] , by.x = "LibM" , by.y = "Lib")

sam_otu <- Otu2[,-c(1:5)]
sam_otu1 = aggregate(. ~  Samplenumber, data = sam_otu, sum)  #some samples are repeted in the lab because of low concentration of DNA so we merged them
#sample
sample = sampleinfo[!duplicated(sampleinfo$Samplenumber) , ]  # now remoe duplate sample names
rownames(sample) <- sample$Samplenumber
rownames(sam_otu1) <- sam_otu1$Samplenumber
sam_otu2 = sam_otu1[-1]
#select and save 0 read samples
sam_lessReads = sam_otu2[rowSums(sam_otu2)==0,]
sam_lessReads1 = merge(x =sam_lessReads[,c(1,ncol(sam_lessReads))] , y=sample , by.x="row.names",by.y="row.names")
sam_lessReads1  = sam_lessReads1[,-c(2,2)]
#remove 0 read samples
sam_otu2 = sam_otu2[!rowSums(sam_otu2)==0,] # use this table for Ra log10 and raw reads saving info
#save abundance tabke with samples using sam_otu2
sample1 = sample[(sample$Samplenumber %in% rownames(sam_otu2)),]
rownames(sample1) <- sample1$Samplenumber
####
sam_otu3 = cbind(TotalReads=rowSums(sam_otu2),numOTUs=ncol(sam_otu2),sam_otu2)
cleansam_otu = merge(x = sample1,y = sam_otu3 , by= "row.names")
cleansam_otu = cleansam_otu[-1] ##after removing unclassified otu st phylum level and samples with less than 100 reads
cleansam_otu = cleansam_otu[order(cleansam_otu$TotalReads,decreasing = TRUE),]
taxonomy = otu_taxa[(otu_taxa$OTU %in% colnames(sam_otu3)),]
taxonomy = taxonomy[-ncol(taxonomy)]
write.table(cleansam_otu,file = "Data/DataClean/PV9Otu.txt",quote = FALSE,sep = "\t",row.names = FALSE)
write.table(taxonomy,file = "Data/DataClean/PV9Taxa.txt",quote = FALSE , sep = "\t" , row.names = FALSE)
#write.table(sam_lessReads1,file = "Data/DataClean/PV9_0readsamples.txt",quote = FALSE , sep = "\t" , row.names = FALSE)
#write.table(Unclassifiedotus,file = "Data/DataClean/PV9Unclaasified.txt",quote = FALSE , sep = "\t" , row.names = FALSE)
print('done!')

[1] "done!"



#### Fungal table preprocessing


In [19]:
Otu = read.table("data/MothurOutput/FITS2Otu.txt", header = T , check.names = F , stringsAsFactors = F)
taxa = read.table("data/MothurOutput/FITS2Taxa.txt", header = T , check.names = F , stringsAsFactors = F)
taxa$Taxonomy = gsub("\\s*\\([^\\)]+\\)","",as.character(taxa$Taxonomy))
otu_taxa <- taxa %>% separate("Taxonomy" , c("Kingdom" , "Phylum" , "Class" , "Order" , "Family" , "Genus" ,"Species" , "Strain" ) , ";" , remove = FALSE)
Unclassifiedotus = subset(otu_taxa,otu_taxa$Phylum=="k__Fungi_unclassified" | otu_taxa$Phylum=="p__unclassified_Fungi") #remove all the otus tat are not classified in phylum level from taxonomy table
Otu1 = Otu[,!(colnames(Otu) %in% Unclassifiedotus$OTU)] #remove remove all the otus tat are not classified in phylum level from otu table
#Otu1 = Otu
sampleinfo <- read.table("data/metadata.txt" , header = T , check.names = F , stringsAsFactors = F) # mapping file containes informatio about data, lab part
#---------------------------------------------------------------------------------------------
Otu2 <- Otu1%>%separate("Group" , c("Loci" , "LibM") , remove = FALSE)
Otu2 = merge(Otu2,sampleinfo[c("Lib","Samplenumber")] , by.x = "LibM" , by.y = "Lib")
sam_otu <- Otu2[,-c(1:5)]
sam_otu1 = aggregate(. ~  Samplenumber, data = sam_otu, sum)  #some samples are repeted in the lab because of low concentration of DNA so we merged them
#sample
sample = sampleinfo[!duplicated(sampleinfo$Samplenumber) , ]  # now remoe duplate sample names
rownames(sample) <- sample$Samplenumber
rownames(sam_otu1) <- sam_otu1$Samplenumber
sam_otu2 = sam_otu1[-1]
#select and save 0 read samples
sam_lessReads = sam_otu2[rowSums(sam_otu2)==0,]
sam_lessReads1 = merge(x =sam_lessReads[,c(1,ncol(sam_lessReads))] , y=sample , by.x="row.names",by.y="row.names")
sam_lessReads1  = sam_lessReads1[,-c(2,2)]
#remove 0 read samples
sam_otu2 = sam_otu2[!rowSums(sam_otu2)==0,] # use this table for Ra log10 and raw reads saving info
#save abundance tabke with samples using sam_otu2
sample1 = sample[(sample$Samplenumber %in% rownames(sam_otu2)),]
rownames(sample1) <- sample1$Samplenumber
####
sam_otu3 = cbind(TotalReads=rowSums(sam_otu2),numOTUs=ncol(sam_otu2),sam_otu2)
cleansam_otu = merge(x = sample1,y = sam_otu3 , by= "row.names")
cleansam_otu = cleansam_otu[-1] ##after removing unclassified otu st phylum level and samples with less than 100 reads
cleansam_otu = cleansam_otu[order(cleansam_otu$TotalReads,decreasing = TRUE),]
taxonomy = otu_taxa[(otu_taxa$OTU %in% colnames(sam_otu3)),]
taxonomy = taxonomy[-ncol(taxonomy)]
write.table(cleansam_otu,file = "Data/DataClean/FITS2Otu.txt",quote = FALSE,sep = "\t",row.names = FALSE)
write.table(taxonomy,file = "Data/DataClean/FITS2Taxa.txt",quote = FALSE , sep = "\t" , row.names = FALSE)
#write.table(sam_lessReads1,file = "Data/DataClean/FITS2_0readsamples.txt",quote = FALSE , sep = "\t" , row.names = FALSE)
#write.table(Unclassifiedotus,file = "Data/DataClean/FITS2Unclaasified.txt",quote = FALSE , sep = "\t" , row.names = FALSE)

print('done!')

[1] "done!"
